# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

## TODO 

- CHECK CORRECTNESS!
- Use buffer for creating bosonic matrices

Then I can immediately start repeating exercises of 0+1


## Improvements

- Only make matrix of Bosons tensored with fermions (better PS representation)
- State ordering option for b0-f0-b1-f1-b2-f2 OR b0-b1-b2-f0-f1-f2

## "Parameters"

- Potential
- (NEW) Lattice Spacing & Sites
- (NEW) Boundary conditions
- Bosonic dof cutoff
- (NEW) State order - for efficiencies sake.
- Binary Encoding


## Plan 

- Exact diagonalization of Hamiltonian - spectrum vs "parameters"
- VQE in QISKIT - quantum procedure
- Machine Learning ground state - compare to old studies
- Gate count for trotter steps

In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *


In [23]:
bosonNI=sp.expand( p[n]**2/(2*aLat) + (aLat/2)*((q[n+1]-q[n-1])/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(q[n])**2 + aLat*V(q[n])*(q[n+1]-q[n-1])/(2*aLat) )
fermionNI=sp.expand( (xd[n]*x[n+1]+xd[n+1]*x[n])/(2*aLat) )
fermionI=sp.expand( (-1)**n*sp.diff(V(q[n]),q[n])*(xd[n]*x[n]-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q[n])**2/2 + V(q[n])*q[n + 1]/2 - V(q[n])*q[n - 1]/2 + q[n + 1]**2/16 - q[n + 1]*q[n - 1]/8 + q[n - 1]**2/16 + p[n]**2/(2*a)

In [4]:
fermionNI + fermionI

(-1)**n*Derivative(V(q[n]), q[n])*\chi[n]*\chi^{\dagger}[n] - 0.5*(-1)**n*Derivative(V(q[n]), q[n]) + \chi[n + 1]*\chi^{\dagger}[n]/(2*a) + \chi[n]*\chi^{\dagger}[n + 1]/(2*a)

In [5]:
N=3
aVal=1

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {q[-1]: q[N-1], q[N]: q[0],
                          x[N]: x[0], xd[N]: xd[0],
                         }
elif bcType == 'dirichlet':
    boundaryConditions = {q[-1]: 0, q[N]: 0,
                          x[N]: 0, xd[N]: 0
                         }

In [6]:
ham=0
totHam=bosonNI+bosonI+fermionNI+fermionI
for i in range(0,N):
    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

def potential(n):
    return 0

for n in range(0,N):
    ham=ham.subs(V(q[n]),potential(n))
ham=ham.subs(aLat,aVal).simplify().expand()
ham

\chi[0]*\chi^{\dagger}[1]/2 + \chi[0]*\chi^{\dagger}[2]/2 + \chi[1]*\chi^{\dagger}[0]/2 + \chi[1]*\chi^{\dagger}[2]/2 + \chi[2]*\chi^{\dagger}[0]/2 + \chi[2]*\chi^{\dagger}[1]/2 + p[0]**2/2 + p[1]**2/2 + p[2]**2/2 + q[0]**2/8 - q[0]*q[1]/8 - q[0]*q[2]/8 + q[1]**2/8 - q[1]*q[2]/8 + q[2]**2/8

In [7]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [8]:
m=1

HOdofSubs = {}
for i in range(0,N):
    HOdofSubs[q[i]] = 0.5*sp.sqrt(2/m)*(a[i] + adag[i])
    HOdofSubs[p[i]] = complex(0,1)*sp.sqrt(2*m)*(adag[i] - a[i])/2
    
hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam

\chi[0]*\chi^{\dagger}[1]/2 + \chi[0]*\chi^{\dagger}[2]/2 + \chi[1]*\chi^{\dagger}[0]/2 + \chi[1]*\chi^{\dagger}[2]/2 + \chi[2]*\chi^{\dagger}[0]/2 + \chi[2]*\chi^{\dagger}[1]/2 - 0.1875*a[0]**2 - 0.0625*a[0]*a[1] - 0.0625*a[0]*a[2] + 0.625*a[0]*a^{\dagger}[0] - 0.0625*a[0]*a^{\dagger}[1] - 0.0625*a[0]*a^{\dagger}[2] - 0.1875*a[1]**2 - 0.0625*a[1]*a[2] - 0.0625*a[1]*a^{\dagger}[0] + 0.625*a[1]*a^{\dagger}[1] - 0.0625*a[1]*a^{\dagger}[2] - 0.1875*a[2]**2 - 0.0625*a[2]*a^{\dagger}[0] - 0.0625*a[2]*a^{\dagger}[1] + 0.625*a[2]*a^{\dagger}[2] - 0.1875*a^{\dagger}[0]**2 - 0.0625*a^{\dagger}[0]*a^{\dagger}[1] - 0.0625*a^{\dagger}[0]*a^{\dagger}[2] - 0.1875*a^{\dagger}[1]**2 - 0.0625*a^{\dagger}[1]*a^{\dagger}[2] - 0.1875*a^{\dagger}[2]**2

In [9]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N)

In [10]:
hamMat

array([[ 3.25      +0.j, -0.25      +0.j, -0.26516504+0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       [ 0.375     +0.j,  3.25      +0.j, -0.35355338+0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       [-0.26516504+0.j,  0.53033008+0.j,  1.25      +0.j, ...,
         0.        +0.j,  0.        +0.j,  0.        +0.j],
       ...,
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
         3.        +0.j,  0.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
         0.        +0.j,  3.        +0.j,  0.        +0.j],
       [ 0.        +0.j,  0.        +0.j,  0.        +0.j, ...,
         0.        +0.j,  0.        +0.j,  3.        +0.j]])

In [11]:
type(hamMat[2,0])

numpy.complex128

In [16]:
np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])

array([0.21453752+0.j        , 0.21453752+0.j        ,
       0.21453752+0.j        , 2.2623823 +0.j        ,
       2.2623823 +0.j        , 2.2623823 +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3.        +0.j        , 3.        +0.j        ,
       3. 

In [17]:
from matrix_to_ps import *
from binary_encodings import *

In [18]:
hamMat.shape

(512, 512)

In [19]:
hamPS=matrix_to_pauli_strings(hamMat, standard_encode)

In [20]:
hamPS=sp.nsimplify(hamPS, tolerance=1e-10)

In [22]:
#sp.simplify(sp.expand(hamPS))